# 🔽 Extração e Preparação do Dataset `trn.json` do Google Drive
Este notebook realiza as seguintes etapas:
1. Instala o pacote `gdown` caso não esteja instalado;
2. Baixa um arquivo `.zip` do Google Drive;
3. Extrai apenas o arquivo `trn.json` de dentro do `.zip`;
4. Remove o `.zip` para economizar espaço em disco;
5. Converte o `trn.json` para o formato `.jsonl` com campos `prompt` e `completion`.


In [7]:
# Instala o gdown automaticamente se necessário
import subprocess, sys

try:
    import gdown
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
    import gdown

## 📁 Baixando o ZIP do Google Drive
Vamos usar o `gdown` para baixar o arquivo `.zip` diretamente de um link público do Google Drive.


In [8]:
zip_file_id = "12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK"
zip_output = "dados.zip"

# Baixa o arquivo ZIP
gdown.download(f"https://drive.google.com/uc?id={zip_file_id}", zip_output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=50dc36d2-79d7-43f1-a96f-d39a3c422022
To: c:\Users\mathe\Documents\FIAP\3-OPENAI\Fase3-Grupo9\dados.zip
100%|██████████| 890M/890M [00:15<00:00, 56.9MB/s] 


'dados.zip'

## 📦 Extraindo apenas o `trn.json` do ZIP
Usamos o módulo `zipfile` da biblioteca padrão para extrair somente o arquivo desejado.


In [9]:
import zipfile
import os
import shutil
import gzip

caminho_zip = "dados.zip"
texto_desejado = "trn.json"
extraido = None
nome_final = None

# Etapa 1: Abrir e localizar arquivo com 'trn.json'
with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
    for arquivo in zip_ref.namelist():
        if texto_desejado in os.path.basename(arquivo):
            zip_ref.extract(arquivo)
            nome_extraido = os.path.basename(arquivo)
            shutil.move(arquivo, nome_extraido)
            extraido = nome_extraido
            print(f"✅ Arquivo extraído: {extraido}")
            break

# Etapa 2: Descompactar .gz se necessário
if extraido:
    if extraido.endswith('.gz'):
        nome_final = extraido.replace('.gz', '')
        with gzip.open(extraido, 'rb') as f_in, open(nome_final, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
        os.remove(extraido)
        print(f"📂 Arquivo GZ descompactado para: {nome_final}")
    elif extraido.endswith('.json'):
        nome_final = extraido
        print(f"📄 Arquivo JSON pronto para uso: {extraido}")
    else:
        print(f"⚠️ Extensão desconhecida: {extraido}")
else:
    print("❌ Nenhum arquivo contendo 'trn.json' foi encontrado.")

# Etapa 3: Limpeza de arquivos e pastas desnecessárias (exceto sample_data)
if nome_final:
    for item in os.listdir():
        if item != nome_final and not item.endswith('.ipynb') and item != 'sample_data' and item != 'llm':
            item_path = os.path.join(os.getcwd(), item)
            try:
                if os.path.isfile(item_path):
                    os.remove(item_path)
                    print(f"🗑️ Arquivo removido: {item}")
                elif os.path.isdir(item_path):
                    shutil.rmtree(item_path)
                    print(f"🗑️ Pasta removida: {item}")
            except Exception as e:
                print(f"⚠️ Erro ao remover {item}: {e}")


✅ Arquivo extraído: trn.json.gz
📂 Arquivo GZ descompactado para: trn.json
⚠️ Erro ao remover .git: [WinError 5] Acesso negado: 'c:\\Users\\mathe\\Documents\\FIAP\\3-OPENAI\\Fase3-Grupo9\\.git\\objects\\92\\3c2416c1fb60ad5ce6b4c2e4982a6c33a81941'
🗑️ Arquivo removido: dados.zip
🗑️ Arquivo removido: dataset_preparado.jsonl
🗑️ Pasta removida: LF-Amazon-1.3M


## 📄 Função para preparar o JSONL
Transformamos o `trn.json` em um arquivo `.jsonl` contendo `prompt` e `completion`, ideal para fine-tuning de modelos de linguagem.


In [10]:
# 🛠️ Função para preparar o dataset em formato jsonl

import json
import html

def preparar_dataset_jsonl(caminho_entrada, caminho_saida):
    phrase_to_remove = "--This text refers to an out of print or unavailable edition of this title."
    contador = 0
    ignorados_total = 0
    erros_json = 0
    titulos_vazios = 0
    descricoes_vazias = 0

    with open(caminho_entrada, 'r', encoding='utf-8') as f_in, \
        open(caminho_saida, 'w', encoding='utf-8') as f_out:

        for linha in f_in:
            try:
                item = json.loads(linha)
                titulo = item.get("title", "").strip()
                descricao = item.get("content", "").strip()

                if not titulo:
                    titulos_vazios += 1
                    ignorados_total += 1
                    continue

                if not descricao:
                    descricoes_vazias += 1
                    ignorados_total += 1
                    continue

                # Remove frase indesejada e converte entidades HTML
                descricao = descricao.replace(phrase_to_remove, "")
                descricao = html.unescape(descricao)

                prompt = f"O que é {titulo}?"
                completion = descricao
                f_out.write(json.dumps({
                    "prompt": prompt,
                    "completion": completion
                }, ensure_ascii=False) + '\n')
                contador += 1

            except json.JSONDecodeError:
                erros_json += 1
                ignorados_total += 1

    print("✅ Dataset gerado com sucesso!")
    print(f"📄 Linhas válidas: {contador}")
    print(f"🚫 Linhas ignoradas: {ignorados_total}")
    print(f"   └─ 🧨 Erros de JSON: {erros_json}")
    print(f"   └─ ❌ Títulos vazios: {titulos_vazios}")
    print(f"   └─ ❌ Descrições vazias: {descricoes_vazias}")
    print(f"💾 Arquivo salvo em: {caminho_saida}")

## ▶️ Executando a Preparação dos Dados


In [11]:
CAMINHO_JSON = "trn.json"
SAIDA_JSONL = "dataset_preparado.jsonl"
preparar_dataset_jsonl(CAMINHO_JSON, SAIDA_JSONL)

✅ Dataset gerado com sucesso!
📄 Linhas válidas: 1390403
🚫 Linhas ignoradas: 858216
   └─ 🧨 Erros de JSON: 0
   └─ ❌ Títulos vazios: 126834
   └─ ❌ Descrições vazias: 731382
💾 Arquivo salvo em: dataset_preparado.jsonl


## ▶️ Verificando a quantidade de caracteres por linhas e max lenght token.

In [17]:
from transformers import AutoTokenizer

MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def analisar_tokens_jsonl(caminho_arquivo, tokenizer, top_n=100):
    tamanhos = []
    linhas = []
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        for linha in f:
            texto = linha.strip()
            tokens = tokenizer.encode(texto)
            tamanhos.append(len(tokens))
            linhas.append(texto)
    print("\nResumo:")
    print(f"Total de linhas: {len(tamanhos)}")
    print(f"Máximo: {max(tamanhos)}")
    print(f"Mínimo: {min(tamanhos)}")
    print(f"Média: {sum(tamanhos)/len(tamanhos):.2f}")
    print(f"Percentil 95: {sorted(tamanhos)[int(0.95*len(tamanhos))-1]}")
    print(f"Média de tokens por linha: {sum(tamanhos)/len(tamanhos):.2f}")

    # Linha com mais tokens
    idx_max = tamanhos.index(max(tamanhos))
    print("\nLinha com mais tokens:")
    # print(linhas[idx_max])
    print(f"Total de tokens nessa linha: {tamanhos[idx_max]}")

    # Top N linhas com mais tokens
    # linhas_tokens = sorted(zip(linhas, tamanhos), key=lambda x: x[1], reverse=True)
    # print(f"\nTop {top_n} linhas com mais tokens:")
    # for i, (linha, num_tokens) in enumerate(linhas_tokens[:top_n], 1):
    #     print(f"{i:03d} - {num_tokens} tokens: {linha}")

    return  linhas, tamanhos

# Exemplo de uso:
linhas, tamanhos = analisar_tokens_jsonl(SAIDA_JSONL, tokenizer)


Resumo:
Total de linhas: 1390403
Máximo: 60624
Mínimo: 15
Média: 163.88
Percentil 95: 406
Média de tokens por linha: 163.88

Linha com mais tokens:
Total de tokens nessa linha: 60624


In [ ]:
# Conta quantas linhas têm mais de 2048 tokens
linhas_acima_2048 = [linha for linha, num_tokens in zip(linhas, tamanhos) if num_tokens > 2048]
quantidade = len(linhas_acima_2048)
print(f"Quantidade de linhas com mais de 2048 tokens: {quantidade}")

# (Opcional) Exibe as linhas
for i, linha in enumerate(linhas_acima_2048, 1):
    print(f"{i:03d}: {linha[:80]}...")  # Mostra só os 80 primeiros caracteres para não poluir a saída

# Salva em um novo arquivo JSONL (arquivo com as que possuem mais de 2048 tokens)
with open('saida_filtrada.jsonl', 'w', encoding='utf-8') as f:
    for linha in linhas_acima_2048:
        f.write(linha + '\n')

print(f"Arquivo 'saida_filtrada.jsonl' salvo com {len(linhas_acima_2048)} linhas.")

Quantidade de linhas com mais de 2048 tokens: 1433
001: {"prompt": "O que é Computer Graphics Using Open GL (2nd Edition)?", "completion...
002: {"prompt": "O que é To Read Literature?", "completion": "To the Student and to t...
003: {"prompt": "O que é The Poetry of Black America: Anthology of the 20th Century?"...
004: {"prompt": "O que é The Last Boy: Mickey Mantle and the End of America's Childho...
005: {"prompt": "O que é Killer Move: A Novel?", "completion": "“Michael Marshall...i...
006: {"prompt": "O que é Eyes Wide Open: A Novel?", "completion": "Amazon Exclusive: ...
007: {"prompt": "O que é Jesus: A Pilgrimage?", "completion": "“James Martin’s riveti...
008: {"prompt": "O que é Pioneers of American Landscape Design (Professional Architec...
009: {"prompt": "O que é Beautiful Built-ins:  Plans for Designing with Stock Cabinet...
010: {"prompt": "O que é Artful Making: What Managers Need to Know About How Artists ...
011: {"prompt": "O que é Handbook of Software Quality Assur

## ▶️ Removendo as linhas que possuem mais de 2048 tokens para o modelo unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit

In [ ]:
# Conta quantas linhas têm mais de 2048 tokens
linhas_acima_2048 = [linha for linha, num_tokens in zip(linhas, tamanhos) if num_tokens > 2048]
quantidade = len(linhas_acima_2048)
print(f"Quantidade de linhas com mais de 2048 tokens: {quantidade}")

# (Opcional) Exibe as linhas
for i, linha in enumerate(linhas_acima_2048, 1):
    print(f"{i:03d}: {linha[:80]}...")  # Mostra só os 80 primeiros caracteres para não poluir a saída

# Salva em um novo arquivo JSONL (arquivo com as que possuem mais de 2048 tokens)
with open('saida_filtrada.jsonl', 'w', encoding='utf-8') as f:
    for linha in linhas_acima_2048:
        f.write(linha + '\n')

print(f"Arquivo 'saida_filtrada.jsonl' salvo com {len(linhas_acima_2048)} linhas.")

Quantidade de linhas com mais de 2048 tokens: 1433
001: {"prompt": "O que é Computer Graphics Using Open GL (2nd Edition)?", "completion...
002: {"prompt": "O que é To Read Literature?", "completion": "To the Student and to t...
003: {"prompt": "O que é The Poetry of Black America: Anthology of the 20th Century?"...
004: {"prompt": "O que é The Last Boy: Mickey Mantle and the End of America's Childho...
005: {"prompt": "O que é Killer Move: A Novel?", "completion": "“Michael Marshall...i...
006: {"prompt": "O que é Eyes Wide Open: A Novel?", "completion": "Amazon Exclusive: ...
007: {"prompt": "O que é Jesus: A Pilgrimage?", "completion": "“James Martin’s riveti...
008: {"prompt": "O que é Pioneers of American Landscape Design (Professional Architec...
009: {"prompt": "O que é Beautiful Built-ins:  Plans for Designing with Stock Cabinet...
010: {"prompt": "O que é Artful Making: What Managers Need to Know About How Artists ...
011: {"prompt": "O que é Handbook of Software Quality Assur

In [ ]:
# Filtra as linhas com mais 2048 tokens
linhas_filtradas = [linha for linha, num_tokens in zip(linhas, tamanhos) if num_tokens <= 2048]

# Salva em um novo arquivo JSONL
with open('dataset_preparado_remove_outliers.jsonl', 'w', encoding='utf-8') as f:
    for i, linha in enumerate(linhas_filtradas):
        if i < len(linhas_filtradas) - 1:
            f.write(linha + '\n')
        else:
            f.write(linha)  # última linha, sem \n

print(f"Arquivo 'dataset_preparado_remove_outliers.jsonl' salvo com {len(linhas_filtradas)} linhas.")

Arquivo 'dataset_preparado_remove_outliers.jsonl' salvo com 1388970 linhas.
